# Preprocessing: BNP01B


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BNP01B'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [5]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")

# save raw as bp cuz it is already rereferenced
raw_clean_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Opening raw data file /home/brooke/pacman/raw_data/BNP01B/ieeg/BNP01B_notched_filtered_clean_ieeg.fif...
Isotrak not found
    Range : 0 ... 268499 =      0.000 ...  1073.996 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BNP01B/ieeg/BNP01B_raw_clean_ieeg.fif...
Isotrak not found
    Range : 0 ... 268499 =      0.000 ...  1073.996 secs
Ready.
Writing /home/brooke/pacman/preprocessing/BNP01B/ieeg/BNP01B_bp_raw_clean_data.fif


/tmp/ipykernel_218509/229302221.py:10: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BNP01B/ieeg/BNP01B_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_clean_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/BNP01B/ieeg/BNP01B_bp_raw_clean_data.fif
[done]


## Epoching the data 

## Last Away

In [15]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,trial_numeric,TrialType,sample,sample_before,event
0,2,14,4381,0,1
1,3,11,6351,0,1
2,4,10,7261,0,1
3,5,5,8151,0,1
4,6,15,9361,0,1
...,...,...,...,...,...
169,175,20,216830,0,1
170,176,10,218000,0,1
171,177,12,219100,0,1
172,178,19,220280,0,1


In [16]:
# filter to good epochs and fix trial indexing
last_away_data['trial_numeric'] = last_away_data['trial_numeric'] - 1

# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [17]:
# epoch the data
last_away_epochs = mne.Epochs(bp_raw_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Not setting metadata
174 matching events found
No baseline correction applied
0 projection items activated
Loading data for 174 events and 2501 original time points ...
0 bad epochs dropped
Loading data for 1 events and 2501 original time points ...
Loading data for 174 events and 2501 original time points ...


/tmp/ipykernel_218509/3409931828.py:7: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BNP01B/ieeg/BNP01B_bp_raw_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')
